In [1]:
#импортируем все необходимые библиотеки для дальнейшего анализа
import pandas as pd
import numpy as np
from collections import Counter
import itertools as r 
import psycopg2
import psycopg2.extras 
import random

# Подготовка данных с помощью SQL
---
> При помощи **SQL** подготовим данные с информацией по продажам курсов на каждого пользователя, который приобрел более одного курса. 

 

In [2]:
#чтобы отфильтровать пользователей, которые приобрели более 1-го курса я обралилась к конструкции СТЕ
#в первой части кода я подготовила подзапрос, где вывела уникальных пользователей (group by), 
#купивших (where) более 1-го курса (having)   
#далее, соединив получившийся запрос с двумя основными таблицами, вывела id-пользователей и id-курсов
def get_data():
    query ='''
    WITH users_grouped AS
( SELECT  c.user_id,
        COUNT (ci.resource_id) AS courses_counted
    FROM final.carts c
    JOIN final.cart_items ci
    ON c.id = ci.cart_id 
    WHERE ci.resource_type = 'Course' AND c.state = 'successful' 
    GROUP BY  c.user_id
    HAVING COUNT (DISTINCT ci.resource_id) > 1)
SELECT  DISTINCT ug.user_id,
        ci2.resource_id
FROM final.carts AS c2
JOIN final.cart_items AS ci2
ON ci2.cart_id = c2.id
JOIN users_grouped AS ug
ON c2.user_id = ug.user_id
WHERE c2.state = 'successful'
    AND ci2.resource_type = 'Course'
ORDER BY ug.user_id, ci2.resource_id
    '''
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' \
    host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
df_courses = pd.DataFrame(get_data())

## Итоговая таблица `COURSES`

In [3]:
df_courses.head()

,user_id,resource_id
0,51,516
1,51,1099
2,6117,356
3,6117,357
4,6117,1125


# Обработка данных 
---
> Чтобы грамотно построить реккомендационную систему, нам необходимо привести данные к нужному виду. Как исход, мы должны получить **список уникальных курсов и словарь**, где ключами будут являться пары из списков, а значениями частотность.

Посмотрим на **количество уникальных курсов**, которые получились у нас в датасете выше: 

In [4]:
#создаем список уникальных курсов, который далее нам понадобится для создания итоговой таблицы с реккомендациями 
unq_courses = df_courses['resource_id'].unique()
print('Всего различных курсов, для которых стоит построить реккомендации:', len(unq_courses))

Всего различных курсов, для которых стоит построить реккомендации: 126


Создаем датасет со списком всех **уникальных курсов, в разрезе по пользователю,** в последнем столбце (set).

In [5]:
df_usr_crs = df_courses.groupby('user_id')['resource_id'].agg(
        ['nunique', set]
).sort_values(by='nunique', ascending=False)
df_usr_crs.head()

,nunique,set
user_id,,
694146,19,"{515, 1156, 776, 523, 907, 908, 909, 670, 552,..."
953401,18,"{514, 515, 741, 518, 519, 551, 552, 490, 523, ..."
1050532,18,"{864, 551, 552, 489, 679, 363, 523, 776, 366, ..."
992470,17,"{514, 515, 517, 518, 519, 551, 489, 552, 523, ..."
722252,17,"{514, 515, 357, 551, 489, 490, 523, 829, 908, ..."


Далее приступим к разбивкe курсов на возможные **пары** по каждому пользователю:

In [6]:
#применяем к столбцу 'set' lambda-функцию с методом combinations(), 
#и получаем уникальные комбинации из элементов множества
course_pairs = df_usr_crs['set'].apply(lambda x: list(r.combinations(sorted(x), 2)))
course_pairs[:10]

user_id
694146     [(489, 490), (489, 515), (489, 523), (489, 552...
953401     [(490, 502), (490, 514), (490, 515), (490, 518...
1050532    [(363, 366), (363, 489), (363, 502), (363, 511...
992470     [(489, 502), (489, 514), (489, 515), (489, 517...
722252     [(357, 366), (357, 489), (357, 490), (357, 502...
109035     [(517, 523), (517, 551), (517, 564), (517, 571...
565025     [(366, 517), (366, 519), (366, 523), (366, 551...
698038     [(489, 504), (489, 515), (489, 523), (489, 552...
1017561    [(363, 489), (363, 502), (363, 511), (363, 515...
1139935    [(489, 502), (489, 504), (489, 514), (489, 515...
Name: set, dtype: object

Далее нам понадобятся только **все пары**, полученные в предыдущем действии, в виде списка:

In [7]:
#создаем пустой список и в него, с помощью итерации, добавляем кортежи с парами id-курсов
courses_list = []
for crs in course_pairs.values:
    for elem in crs: 
        courses_list.append(elem)
courses_list[:10]

[(489, 490),
 (489, 515),
 (489, 523),
 (489, 552),
 (489, 564),
 (489, 566),
 (489, 670),
 (489, 764),
 (489, 776),
 (489, 809)]

In [8]:
#при помощи счетчика Counter() подсчитаем сколько раз нам встречаются различные пары  
counter_courses = Counter(courses_list)
#преобразуем получившийся объект в словарь с которым будем работать в дальнейшем 
dict_counter = dict(counter_courses)

В итоге мы получаем ***словарь `"dict_counter"`***, который имеет следующий вид: 

---
>***{(id-course_1, id_course_2): count, ..}*** - где ключом является список из пар курсов, а значением выступает количество встреч данной пары в предыдущем списке (courses_list)

# Подготовка к составлению итоговой таблицы
---
>Необходимо определить **минимальное значение** частотности курсов для реккомендации и **подготовить данные** для составления итоговой таблицы. 

Рассмотрим **частотность** встречающихся редких пар курсов: 

In [9]:
#создаем датафрейм из значений в словаре, чтобы посмотреть на частотность встречающихся редких пар курсов
df = pd.DataFrame(list(dict_counter.items()), columns =['pairs', 'frequency']).sort_values(by= 'frequency', \
                                                                          ascending = False).reset_index(drop=True)
df['frequency'].value_counts().head(10)

1     1185
2      578
3      366
4      259
5      202
7      148
6      148
8       95
10      87
9       80
Name: frequency, dtype: int64

Посмотрим на **статистику:**

In [10]:
df['frequency'].describe()

count    3989.000000
mean       10.031838
std        26.355998
min         1.000000
25%         1.000000
50%         3.000000
75%         9.000000
max       797.000000
Name: frequency, dtype: float64

Видим, что **чаще всего встречается единоразовая покупка пар курсов.** 
Также заметно, что количество продаж распределено не нормально, **есть выбросы** в виде количества проданных раз популярных курсов. 
**Медиана равна 3**. Столько в среднем встречаются уникальные пары в корзине. 

Теперь осталось определиться минимальным значением для выбора реккомендаций. 

Для начала нужно определиться с конкретной целью введения фичи. Исходя из слов продакт-менеджера мы выяснили, что он хочет чтобы продавались **новые курсы**, тем самым **поднять средний чек.**

**Тут два момента:**
* с одной стороны мы **не можем сказать**, как будут **продаваться** новые курсы (ведь они явно уступают по продажам популярным курсам),
* второй момент, если **заменять** самыми популярными курсами пары, которые не так часто встречаются, то можно **не попасть** в интересы покупателя (ведь пары могут дать некую гарантию, что курсы взаимосвязаны, но не во всех случаях).
---
Поэтому я хочу взять за минимальную границу что-то среднее между первым и третьим квартилями : **5**

Столько раз должна попадаться пара при покупке курсов для ее реккомендации.

In [11]:
#записываем в переменную значение
min_value = 5 

Посмотрим на количество покупок по каждому курсу и выделим **10-ку лучших.** 

In [12]:
#делаем группировку по id-курсам и считаем их же количество, сортируем по убыванию частотности 
group_rating = df_courses.groupby('resource_id')['resource_id'].count().sort_values(ascending=False)[:10]
group_rating

resource_id
551    2935
566    2342
515    1311
489    1125
490    1084
523    1053
514     983
794     875
745     829
570     720
Name: resource_id, dtype: int64

In [13]:
#запишем 10-ку самых востребованных курсов в список, он нам понадобится для создания итоговой таблицы 
top_courses = list(group_rating.index)
top_courses

[551, 566, 515, 489, 490, 523, 514, 794, 745, 570]

Теперь посмотрим на **пары курсов**, которые прдавались чаще всего, выделим топ 10 пар:

In [14]:
#создаем словарь, куда записываем десять пар с максимальным значением по продажам
top_pairs = dict(counter_courses.most_common(10))
#в новый список добавляем только пары 
#данный список нам понадобится для создания итоговой таблицы 
pairs_reccom = []
for pair in top_pairs.keys():
    pairs_reccom.append(pair)
pairs_reccom

[(551, 566),
 (515, 551),
 (489, 551),
 (523, 551),
 (566, 794),
 (489, 515),
 (490, 566),
 (490, 551),
 (570, 752),
 (569, 572)]

# Составление итоговой таблицы
---
> * Необходимо **написать функцию** для создания списка пар под id-курсы с максимальным значением по продажам. Нам нужны всего две пары на один курс. 
> * Создать **итератор**, который будет записывать **реккомендации в итоговую таблицу.**

***Примечание***:

В создании функции предполагается, что будут учавствовать курсы, которые не встречаются в итерируемом словаре и курсы, которые встречаются в словаре парой всего один раз, а для создания реккомендаций нам нужно два курса. Поэтому итоговый список, вместо пропущенных значений, я буду заполнять топовыми курсами по продажам или парами топовых курсов, взависимости от количества пропущенных элементов. 

Приступим к **написанию функции**, которая должна возвращать **список пар с максимальной частотностью** для каждого уникального курса заказчика. 

In [15]:
def recommend(course_id):
    recomm_list = [] #cоздаем список пар с частотностью под определенный запрос(id курса)
    for i in dict_counter.keys():
        #проверяем условие нахождения курса в словаре с парами и частотностью
        if i[0] == course_id: 
            recomm_list.append((i, dict_counter[i]))
        #если курса нет в словаре, добавляем пару самостоятельно, рандомно выбрав и списка пар топовых курсов 
        if course_id not in dict_counter.keys():
            #создаем кортеж, куда записываем рандомно пару из списка топовых 
            rm = random.choice(pairs_reccom)
            #вместо частотности добавлем минимальное значение, чтобы список прошел фильтр в коде ниже
            recomm_list.extend([((course_id, rm[0] ), min_value)])
            recomm_list.extend([((course_id, rm[1]), min_value)])
    #сортируем получившийся список в порядке убывания
    recomm_list = (sorted(recomm_list, key=lambda x: (x[:][1]), reverse = True))
    #проверяем еще одно условие - длину получившегося списка выше, 
    #для дальнейшей корректной работы нам понадобится список состоявший из двух элементов 
    if len(recomm_list) == 1:
        #если длина равна единице, рандомно выбираем курсу пару из списка топовых курсов
        recomm_list.extend([((course_id, random.choice(top_courses)), min_value)])
    #выводим список, длинной в 2 элемента
    return recomm_list[:2]

Приступим к написанию **итератора**, который будет записывать реккомендации в итоговую таблицу. 

Итоговая таблица будет состоять из **трех столбцов**:

>* В первом столбце **курс**, к которому будут записаны две реккомендации (назовем этот курс **основным**), а именно:
>* во втором столбце **реккомендация№1**, это курс который стоит в паре с основным первым в списке по частотности,
>* в третьем столбце **реккомендация№2**, это курс который стоит вторым в паре с оновным в списке по частотности. 

Для курсов, чьи пары по частотности встречаются **слишком редко (менее 5 раз)**, будут записываться **пары топовых курсов** по частотности продаж или **топовые курсы** по продажам, взависимости от ситуации. 

In [16]:
#создаем пустой датафрейм, куда в дальнейшем будем записывать реккомендации  
recommendation_table = pd.DataFrame(columns = ['current_course', 'recommend_1', 'recommend_2'])
for i in unq_courses:
    #проверяем условия, где частотность двух пар должна быть выше заданного мин значения
    if  recommend(i)[0][1] >= min_value and recommend(i)[1][1] >= min_value:
        #записываем в первый столбец текущий курс, и далее в два стобца реккомендации, идущие к данному курсу
        new_row = {'current_course':recommend(i)[0][0][0],\
                   'recommend_1':recommend(i)[0][0][1],\
                   'recommend_2':recommend(i)[1][0][1]}
    elif recommend(i)[0][1] < min_value and recommend(i)[1][1] >= min_value:
        #если одна из частотности двух пар ниже минимального значения, 
        #вместо данной пары записываем рандомно выбранный курс из списка топовых курсов 
        new_row = {'current_course':recommend(i)[0][0][0],\
                   'recommend_1':random.choice(top_courses),\
                   'recommend_2':recommend(i)[1][0][1]}
    elif recommend(i)[0][1] >= min_value and recommend(i)[1][1] < min_value:
        new_row = {'current_course':recommend(i)[0][0][0],\
                   'recommend_1':recommend(i)[0][0][1],\
                   'recommend_2':random.choice(top_courses)}
    #если встретился курс, у которого все пары имеют частотность ниже минимального значения,
    #вместо данных пар в реккомендации записываем рандомно выбранные пары из списка топовых пар
    else:
        rm = random.choice(pairs_reccom)
        new_row = {'current_course':recommend(i)[0][0][0],\
                   'recommend_1':rm[0],\
                   'recommend_2':rm[1]}
    #записываем в датафрейм получившиеся строки
    recommendation_table = recommendation_table.append(new_row, ignore_index = True)

# Финальная таблица `recommendation_table`

In [17]:
recommendation_table

,current_course,recommend_1,recommend_2
0,516,745,553
1,1099,1139,1187
2,356,571,357
3,357,571,1125
4,1125,1186,1144
...,...,...,...
121,902,566,551
122,837,490,572
123,1200,489,551
124,833,523,551


# Выводы:
В рамках проекта нашей задачей являлась **разработка реккомендательной системы** при которой можно будет предлагать клиентам интересные им курсы с целью повышения среднего чека.
Как итог нужно было представить таблицу, в которой курсам будет соответствовать по **две рекомендации.**

В ходе проекта я, проанализировав все данные, пришла к выводу, что **не все курсы продавались в паре** с другими продуктами. И что для таких пар необходимо записывать **пару курсов**(список уникальных топовых пар), которые хорошо продаются.  
Также есть курсы, которые за все время имели только **одну пару** и курсы, которые парами встречались **очень редко**, для таких случаев в реккомендациии записывались **курсы** (список уникальных топовых курсов), которые хорошо продаются.

Таким образом, на мой взгляд, мы удовлетворим запрос менеджера по увеличению среднего чека и не быть "артистом одной песни" (в реккомендации записывались курсы, которые встречались парой от 5 раз)